In [1]:
# Import Modules
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
#import yfinance as yf
from pathlib import Path
import sqlalchemy as sql
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
load_dotenv("./SAMPLE.env")

True

In [3]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [4]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
tickers = ["RYT","XSW","XLK","USRT","XLRE","RWR","BBUS","JMOM","SPMD","MDYG","SLY","SLYG","SPY", "QQQ", "GLD"]
timeframe = "1D"
start_date = pd.Timestamp("2019-10-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-01", tz="America/New_York").isoformat()

In [6]:
df_hist_data = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = 1000
).df

In [7]:
df_hist_data.head()

BBUS                                       GLD  \
                            open     high      low    close volume    open   
time                                                                         
2019-10-01 00:00:00-04:00  53.63  53.6900  52.8100  52.8100  12961  138.07   
2019-10-02 00:00:00-04:00  52.53  52.5300  51.7468  51.8799  16333  140.79   
2019-10-03 00:00:00-04:00  51.89  52.1600  51.3300  52.1600  17298  141.36   
2019-10-04 00:00:00-04:00  52.50  52.7701  52.4900  52.7701  10647  141.48   
2019-10-07 00:00:00-04:00  52.87  53.1400  52.8200  52.8200  11417  141.16   

                                                                ...   XLRE  \
                               high      low   close    volume  ...   open   
time                                                            ...          
2019-10-01 00:00:00-04:00  140.1900  137.800  139.62  10473007  ...  39.30   
2019-10-02 00:00:00-04:00  141.8800  140.462  141.27  10190256  ...  38.95   
2019-10-03 00:00:00-04:00  143.2500  141.290  141.91  10286203  ...  38.76   
2019-10-04 00:00:00-04:00  142.3100  141.140  141.93   8152981  ...  39.28   
2019-10-07 00:00:00-04:00  141.7709  140.240  140.72   5399210  ...  39.30   

                                                               XSW           \
                              high      low  close   volume   open     high   
time                                                                          
2019-10-01 00:00:00-04:00  39.4000  38.8500  38.97  5145503  93.59  93.9836   
2019-10-02 00:00:00-04:00  39.0450  38.5300  38.75  4936233  91.72  91.7200   
2019-10-03 00:00:00-04:00  39.3000  38.7000  39.22  5743093  91.00  92.5346   
2019-10-04 00:00:00-04:00  39.4898  39.2400  39.43  2482671  92.88  93.9600   
2019-10-07 00:00:00-04:00  39.5100  39.1499  39.32  3012752  93.70  94.5381   

                                                  
                               low  close volume  
time                                              
2019-10-01 00:00:00-04:00  92.2100  92.32  30947  
2019-10-02 00:00:00-04:00  90.3001  91.08  24876  
2019-10-03 00:00:00-04:00  89.9100  92.41  39290  
2019-10-04 00:00:00-04:00  92.7883  93.95  45231  
2019-10-07 00:00:00-04:00  93.7000  94.26  13309  

[5 rows x 75 columns]

In [8]:
df_hist_data.tail()

BBUS                                   GLD  \
                            open    high     low  close volume    open   
time                                                                     
2021-09-27 00:00:00-04:00  81.32  81.520  81.121  81.27  36219  163.88   
2021-09-28 00:00:00-04:00  80.67  80.690  79.480  79.56  27047  162.73   
2021-09-29 00:00:00-04:00  79.79  80.120  79.590  79.62  32278  162.35   
2021-09-30 00:00:00-04:00  79.94  80.030  78.720  78.72  25225  162.66   
2021-10-01 00:00:00-04:00  79.09  79.935  78.490  79.56  39710  164.27   

                                                              ...   XLRE  \
                              high      low   close   volume  ...   open   
time                                                          ...          
2021-09-27 00:00:00-04:00  164.365  163.510  163.65  4493388  ...  45.92   
2021-09-28 00:00:00-04:00  162.820  161.730  162.06  8410601  ...  44.87   
2021-09-29 00:00:00-04:00  162.740  160.970  161.36  7813632  ...  45.06   
2021-09-30 00:00:00-04:00  164.950  162.340  164.19  9592918  ...  45.38   
2021-10-01 00:00:00-04:00  164.960  163.885  164.58  6087266  ...  44.79   

                                                               XSW          \
                             high     low  close    volume    open    high   
time                                                                         
2021-09-27 00:00:00-04:00  46.140  45.105  45.13   6621245  178.39  178.94   
2021-09-28 00:00:00-04:00  45.080  44.530  44.87  11755550  175.80  175.80   
2021-09-29 00:00:00-04:00  45.380  45.000  45.18   5669530  172.98  173.47   
2021-09-30 00:00:00-04:00  45.395  44.450  44.47   5820243  172.37  173.27   
2021-10-01 00:00:00-04:00  45.250  44.400  44.93   8033142  172.65  174.93   

                                                     
                                low   close  volume  
time                                                 
2021-09-27 00:00:00-04:00  176.6798  177.98  193398  
2021-09-28 00:00:00-04:00  172.1200  172.12    9133  
2021-09-29 00:00:00-04:00  171.7800  171.80    8595  
2021-09-30 00:00:00-04:00  171.4448  171.63   13834  
2021-10-01 00:00:00-04:00  171.3850  174.62    6558  

[5 rows x 75 columns]

In [9]:
def performance_forecast(etf_df, sim_num=500, term=10):
    
    # Compute MC Confidence Interval for 'term' years
    MC_sim = MCSimulation(
        portfolio_data = etf_df,
        num_simulation = sim_num,
        num_trading_days = 252*term)
    cum_return = MC_sim.summarize_cumulative_return()
    ci_lower = cum_return[8]
    ci_upper = cum_return[9]
    
    return ci_lower, ci_upper

In [10]:
# Load etf list from csv
csv_path = Path('./Resources/etf_list.csv')
etf_list = pd.read_csv(csv_path)['etf']

etf_list_3extra = ["SPY", "QQQ", "GLD"]

# Initialize parameters
etf_final_list = []
ci_lower_10yrs_list = []
ci_upper_10yrs_list = []
ci_lower_20yrs_list = []
ci_upper_20yrs_list = []
ci_lower_30yrs_list = []
ci_upper_30yrs_list = []

# Loop over etf list and compute Confidence Interval for each etf
for i in range(len(etf_list)):
    current_etf = etf_list[i]
    print("Working on ETF: " + current_etf)
    etf_df = df_hist_data.loc[:,[current_etf]]
    # Do 10, 20, 30 years MC for current etf respectively
    print("Computing 10 years return ... ")
    ci_lower_10yrs, ci_upper_10yrs = performance_forecast(etf_df, sim_num=500, term=10)
    print("Computing 20 years return ... ")
    ci_lower_20yrs, ci_upper_20yrs = performance_forecast(etf_df, sim_num=500, term=20)
    print("Computing 30 years return ... ")
    ci_lower_30yrs, ci_upper_30yrs = performance_forecast(etf_df, sim_num=500, term=30)
    # Store results into list
    etf_final_list.append(current_etf)
    ci_lower_10yrs_list.append(ci_lower_10yrs)
    ci_upper_10yrs_list.append(ci_upper_10yrs)
    ci_lower_20yrs_list.append(ci_lower_20yrs)
    ci_upper_20yrs_list.append(ci_upper_20yrs)
    ci_lower_30yrs_list.append(ci_lower_30yrs)
    ci_upper_30yrs_list.append(ci_upper_30yrs)
    
# Loop over 3 extra etf and compute Confidence Interval for each etf
for i in range(len(etf_list_3extra)):
    current_etf = etf_list_3extra[i]
    print("Working on ETF: " + current_etf)
    etf_df = df_hist_data.loc[:,[current_etf]]
    # Do 10, 20, 30 years MC for current etf respectively
    print("Computing 10 years return ... ")
    ci_lower_10yrs, ci_upper_10yrs = performance_forecast(etf_df, sim_num=500, term=10)
    print("Computing 20 years return ... ")
    ci_lower_20yrs, ci_upper_20yrs = performance_forecast(etf_df, sim_num=500, term=20)
    print("Computing 30 years return ... ")
    ci_lower_30yrs, ci_upper_30yrs = performance_forecast(etf_df, sim_num=500, term=30)
    # Store results into list
    etf_final_list.append(current_etf)
    ci_lower_10yrs_list.append(ci_lower_10yrs)
    ci_upper_10yrs_list.append(ci_upper_10yrs)
    ci_lower_20yrs_list.append(ci_lower_20yrs)
    ci_upper_20yrs_list.append(ci_upper_20yrs)
    ci_lower_30yrs_list.append(ci_lower_30yrs)
    ci_upper_30yrs_list.append(ci_upper_30yrs)

# Create ETF performance datafram
etf_performance_df = pd.DataFrame()
etf_performance_df['ETF'] = etf_final_list
etf_performance_df['ci_lower_10yrs'] = ci_lower_10yrs_list
etf_performance_df['ci_upper_10yrs'] = ci_upper_10yrs_list
etf_performance_df['ci_lower_20yrs'] = ci_lower_20yrs_list
etf_performance_df['ci_upper_20yrs'] = ci_upper_20yrs_list
etf_performance_df['ci_lower_30yrs'] = ci_lower_30yrs_list
etf_performance_df['ci_upper_30yrs'] = ci_upper_30yrs_list

etf_performance_df

Working on ETF: RYT
Computing 10 years return ... 
Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/media/xing/DATA61/fintech/Project_Dashboard/MCForecastTools.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,ETF,ci_lower_10yrs,ci_upper_10yrs,ci_lower_20yrs,ci_upper_20yrs,ci_lower_30yrs,ci_upper_30yrs
0,RYT,2.319047,87.087587,12.275791,2740.496510,80.045448,38671.659391
1,XSW,3.831391,185.840732,47.458160,6381.575323,556.790421,273176.333931
2,XLK,3.839963,143.964579,54.062065,9210.080891,800.719806,424996.520080
3,USRT,0.207801,14.133461,0.142961,33.878434,0.084082,141.679678
4,XLRE,0.363407,12.859155,0.279773,68.292971,0.368757,260.359353
5,RWR,0.159932,8.895657,0.097736,27.026801,0.033039,63.076450
6,BBUS,1.407822,34.397452,6.343035,526.829849,40.310061,10052.944692
7,JMOM,1.779034,54.756703,10.683051,768.436939,40.789546,11504.122513
8,SPMD,0.883536,34.757821,1.979439,542.893172,6.415449,4533.399418
9,MDYG,1.033390,32.372989,2.870807,432.752261,12.921456,5600.006241


In [12]:
# Review ETF performance dataframe
etf_performance_df

,ETF,ci_lower_10yrs,ci_upper_10yrs,ci_lower_20yrs,ci_upper_20yrs,ci_lower_30yrs,ci_upper_30yrs
0,RYT,2.319047,87.087587,12.275791,2740.496510,80.045448,38671.659391
1,XSW,3.831391,185.840732,47.458160,6381.575323,556.790421,273176.333931
2,XLK,3.839963,143.964579,54.062065,9210.080891,800.719806,424996.520080
3,USRT,0.207801,14.133461,0.142961,33.878434,0.084082,141.679678
4,XLRE,0.363407,12.859155,0.279773,68.292971,0.368757,260.359353
5,RWR,0.159932,8.895657,0.097736,27.026801,0.033039,63.076450
6,BBUS,1.407822,34.397452,6.343035,526.829849,40.310061,10052.944692
7,JMOM,1.779034,54.756703,10.683051,768.436939,40.789546,11504.122513
8,SPMD,0.883536,34.757821,1.979439,542.893172,6.415449,4533.399418
9,MDYG,1.033390,32.372989,2.870807,432.752261,12.921456,5600.006241
